In [7]:
!pip install pillow 
!pip install matplotlib
!pip install numpy
!pip install qiskit
!pip install qiskit-aer 

In [ ]:
# Dr. Bayntun - 
# The code below is the result for the image with the horizontal lines.
# If you set image_path = the path leading to your own image, it will print the matrix associated with it.

In [11]:
from PIL import Image
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# convert image to binary / black and white 

image_path = r"C:\Users\chris\Desktop\qchorizontal.png"
image = Image.open(image_path).convert('1')
image_data = np.array(image)


# Initialize  14x14 matrix to store 2x2 blocks
result_matrix = np.zeros((14, 14), dtype=object)

# Loop through the 16x16 w/ 2x2
for i in range(14):  # Row 
    for j in range(14):  # Column  
        block_2x2 = image_data[i:i+2, j:j+2]
        result_matrix[i, j] = block_2x2

# making the oracles based on the block
def create_oracle_from_block(block):
    qc = QuantumCircuit(4)  # Create QC
    
    if block[0, 0] == 1:
        qc.x(0)  
    if block[0, 1] == 1:
        qc.x(1)  
    if block[1, 0] == 1:
        qc.x(2)  
    if block[1, 1] == 1:
        qc.x(3)  
            
    
    if block[0, 0] == 1 and block[0, 1] == 1:
        qc.cx(0, 3)
    if block[1, 0] == 1 and block[1, 1] == 1:
        qc.cx(1, 3)

    return qc

# DJ
def deutsch_jozsa(block):
    # create qc
    qc = QuantumCircuit(5, 4)

    #  initalize qubits 
    if block[0, 0] == 1:
        qc.x(0)  # Set qubit 0 to |1⟩ if block[0, 0] is 1
    if block[0, 1] == 1:
        qc.x(1)  # Set qubit 1 to |1⟩ if block[0, 1] is 1
    if block[1, 0] == 1:
        qc.x(2)  # Set qubit 2 to |1⟩ if block[1, 0] is 1
    if block[1, 1] == 1:
        qc.x(3)  # Set qubit 3 to |1⟩ if block[1, 1] is 1
    
    # Initialize the auxiliary qubit to |1⟩
    qc.x(4)
    qc.h(4)
    
    # Apply Hadamard
    qc.h(range(4))
    
    # make Uf
    oracle_circuit = create_oracle_from_block(block)
    
    # Uf
    qc.append(oracle_circuit, range(4))
    
    # Apply Hadamard 
    qc.h(range(4))
    
    # Measure 
    qc.measure(range(4), range(4))
    
    # simulate the QC 
    simulator = AerSimulator()
    transpiled_circuit = transpile(qc, simulator)
    
    # Run the simulation
    result = simulator.run(transpiled_circuit, shots=1).result()
    counts = result.get_counts()
    
    # Return result based on counts
    return 'constant' if '0000' in counts else 'balanced'

# Initialize the 14x14 matrix to store results
results_matrix = np.empty((14, 14), dtype=object)

# Loop over each block in 14x14 matrix
for i in range(14):
    for j in range(14):
        # get the 2x2 block
        block = result_matrix[i, j]
        
        # Run DJ on it 
        result = deutsch_jozsa(block)
        
        # Store result 
        results_matrix[i, j] = result


# Convert constant and balanced to f & e 
for i in range(14):
    for j in range(14):
        if results_matrix[i, j] == 'constant':
            results_matrix[i, j] = 'f'  # Fill
        elif results_matrix[i, j] == 'balanced':
            results_matrix[i, j] = 'e'  # Edge

# Display the final 14x14 grid of 'e' and 'f'
print("14x14 Matrix of 'e' (edge) and 'f' (fill) Results:")
print(results_matrix)

14x14 Matrix of 'e' (edge) and 'f' (fill) Results:
[['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f']]
